# Tunnelling Optimizer

Simulated annealing is a useful optimization algorithm for a wide range of problems, in particular its ability to escape local optima better than hill climbing and gradient descent, its computational cheapness, and its non-reliance on gradients etc. However, it can still struggle with functions with many local optima, as the cooling process causes it to fail to escape. 

In this notebook we use an optimization algorithm inspired by quantum tunnelling, the "tunnelling optimizer". In quantum tunnelling, a particle can "tunnel" out of a potential barrier, which is made more likely if some kind of field is applied to it. Our algorithm gains more "field" the longer it has gone without seeing an improvement, which allows it to take larger steps and aeventuallly escape local optima. The goal is to keep the benefits of simulated annealing while improving performance on tasks wikth many local optima.

In [1]:
import numpy as np
import random as r

## Griewank

Griewank is an interesting optimization problem due to its many local optima and variable dimensions. One interesting area in optimization is how well algorithms perform when there are a large number of dimensions and few objective function calls. Thus, we test 100-dimensional Griewank with 1000 function calls. For more on Griewank: https://www.sfu.ca/~ssurjano/griewank.html

In [2]:
D = 100 #number of dimensions

def griewank(x, numDim = D):
    prodTerm = 1
    for i in range(len(x)):
        prodTerm *= np.cos(x[i] / np.sqrt(i + 1))
    sumTerm = 0
    for i in range(len(x)):
        sumTerm += (x[i] ** 2) / 4000
    output = sumTerm - prodTerm + 1
    return(-output) #since we maximize
ranges = [[-600, 600]] * D

maxFunctionCalls = 1000

## Random Search

Random Search (RS) is computationally cheap and a useful control. If your algorithm doesn't outperform RS, it's back to the drawing board!

In [3]:
class RandomSearch:
    def __init__(
        self,
        ranges
    ):
        self.ranges = ranges
        
    def generateRandomHypothesis(self):
        output = []
        for x in self.ranges:
            output.append(r.uniform(x[0], x[1]))
        return(output)
    
    def optimize(self,
                objectiveFunction,
                numIterations = maxFunctionCalls):
        bestX = None
        bestY = -float("inf")
        for i in range(numIterations):
            newX = self.generateRandomHypothesis()
            newY = objectiveFunction(newX)
            if newY > bestY:
                bestX = newX
                bestY = newY
        return(bestX)
rs = RandomSearch(ranges)
bestX = rs.optimize(griewank)
print(griewank(bestX))

-2051.199905404199


## Simulated Annealing

And of course our main competition, simulated annealing

In [4]:
class SimulatedAnnealing:
    def __init__(
        self,
        ranges
    ):
        self.ranges = ranges
    
    def optimize(self, objectiveFunction, lastX = [], numIterations = maxFunctionCalls, standardDeviation = 0.1, coolingFactor = 0.99):
        currentX = lastX
        while len(currentX) == 0:
            currentX = [r.uniform(c[0], c[1]) for c in ranges]
        currentY = objectiveFunction(currentX)
        heat = 1
        for i in range(numIterations):
            heat = heat * coolingFactor
            newX = [r.gauss(x, standardDeviation) for x in currentX]
            newY = objectiveFunction(newX)
            if newY > currentY:
                currentY = newY
                currentX = newX
            else:
                prob = r.uniform(0, 1)
                if prob > np.e ** -(currentY - newY)/(heat + (10 ** -10)):
                    currentY = newY
                    currentX = newX
        return(currentX)

sa = SimulatedAnnealing(ranges)
bestX = sa.optimize(griewank)
print(griewank(bestX))

-3045.4429570820716


## Tunnelling Optimizer

Here's the fun part, where we test Secretary Optimization and compare to the other algorithms.

In [5]:
class TunnellingOptimizer:
    def __init__(
        self,
        ranges
    ):
        self.ranges = ranges
    
    def optimize(self, objectiveFunction, lastX = [], numIterations = maxFunctionCalls, standardDeviation = 10 ** -3, fieldStrength = 1.01):
        ranges = self.ranges
        dTerm = (ranges[0][1] - ranges[0][0]) * (len(ranges) ** 0.5) * ((6) ** 0.5)
        currentX = lastX
        while len(currentX) == 0:
            currentX = [r.uniform(c[0], c[1]) for c in ranges]
        currentY = objectiveFunction(currentX)
        currentField = 1
        for i in range(numIterations):
            newX = [r.gauss(x, currentField * standardDeviation * dTerm) for x in currentX]
            newY = objectiveFunction(newX)
            if newY >= currentY:
                currentY = newY
                currentX = newX
                currentField = 1
            else:
                currentField = currentField * fieldStrength
        return(currentX)

ta = TunnellingOptimizer(ranges)
bestX = ta.optimize(griewank)
print(griewank(bestX))

-324.59059453076793
